# Product takers and non product takers summary.

Looking into the distribution between product takers and non-product takers.

In [2]:
import pandas as pd
import numpy as np
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy as scipy
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [3]:
from sensorfabric.athena import athena

In [4]:
%set_env AWS_PROFILE=opensci
db = athena(database='menolife', workgroup='amyris', offlineCache=True)

env: AWS_PROFILE=opensci


Number of people who are product takers who have healthid (those who entered symptom data). - 9225

In [9]:
results = db.execQuery("""
    select count(distinct(healthid)) c from product_healthid
""", cached=True)

In [10]:
print(results['c'])

0    9225
Name: c, dtype: object


Percentage of users who reported symptoms that are product takers and non-product takers.

In [19]:
ts = db.execQuery('select count(distinct(uid)) c from symptompp', cached=True)
pt = db.execQuery("""
    select count(distinct(uid)) c from "symptompp"
    where uid in (select "healthid" from "product_healthid") 
""", cached=True)

In [23]:
pd.DataFrame(
    {
        'Total Users' : ts['c'],
        'Product Takers' : pt['c'],
        'PT %' : np.round((pt['c']*100.0)/ts['c'], 2),
        'Non-Product Takers' : ts['c'] - pt['c'],
        'NPT %' : np.round(((ts['c'] - pt['c'])*100.0)/ts['c'], 2)
    }
)

,Total Users,Product Takers,PT %,Non-Product Takers,NPT %
0,25324,4613,18.22,20711,81.78
